In [1]:
# used this to force installation of 'impute' which wasn't available for this version of R
# install.packages("https://cran.r-project.org/src/contrib/Archive/impute/impute_1.26.0.tar.gz", repos = NULL, type="source")
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("pcaMethods")
# install.packages("imputeLCMD")

In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Data Processing/Output')
cur_date = "021424"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)
library(imputeLCMD)
library(vsn)

# reading in files
mRNA_df = data.frame(read_excel("Input/mRNA_Data_041423.xlsx", sheet = 2))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some ma

In [2]:
head(mRNA_df)

Subject_No Subject_ID mRNA    Condensate Burn_Condition Concentration
1 1          F_1        ALDH1A3 C          F              1            
2 1          F_1        ALDH1A3 C          F              1            
3 1          F_1        ALDH1A3 C          S              1            
4 1          F_1        ALDH1A3 C          F              5            
5 1          F_1        ALDH1A3 P          F              1            
6 1          F_1        ALDH1A3 P          S              1            
  Time_Point Replicate ddCT       ddCT_pslog2
1 24         1         -6.6824749 4.480105   
2 72         1         -0.2119125 4.847400   
3 72         1          0.3529920 4.875436   
4 24         1         -0.1184095 4.852078   
5 24         1         -0.2508904 4.845445   
6 24         1          0.9715878 4.905524

Identifying the number of replicates for each sample condition.

In [3]:
count_mRNA_df = mRNA_df %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    mutate(Duplicate_Count = n()) %>% 
    arrange(-Duplicate_Count, mRNA, Subject_No, Condensate, Burn_Condition, Concentration, Time_Point) 

head(count_mRNA_df)

Subject_No Subject_ID mRNA  Condensate Burn_Condition Concentration
1 6          M_6        HMOX1 PBS        PBS            NA           
2 6          M_6        HMOX1 PBS        PBS            NA           
3 6          M_6        HMOX1 PBS        PBS            NA           
4 6          M_6        HMOX1 PBS        PBS            NA           
5 6          M_6        HMOX1 PBS        PBS            NA           
6 6          M_6        HMOX1 PBS        PBS            NA           
  Time_Point Replicate ddCT        ddCT_pslog2 Duplicate_Count
1 24         NA         0.70848207 4.892803    6              
2 24         NA         0.36651641 4.876100    6              
3 24         NA        -0.09090465 4.853452    6              
4 24         NA        -0.37654696 4.839126    6              
5 24         NA         0.72283828 4.893500    6              
6 24         NA         0.37135970 4.876338    6

In [4]:
replicate_count_df = count_mRNA_df %>%
    group_by(Duplicate_Count) %>%
    count() %>%
    arrange(-Duplicate_Count) 

replicate_count_df

Duplicate_Count n   
1 6                  6
2 5                 80
3 4                328
4 3                 96
5 2               1340
6 1               2361

In [5]:
# creating a new column for the replicate number for each condition
replicate_vector = c()
for (i in 1:length(replicate_count_df$n)){
    # calculating number of times to repeat the replicate number from 1 to the count
    replicate_count = replicate_count_df$n[i]/replicate_count_df$Duplicate_Count[i]
    replicate_iteration = rep(1:replicate_count_df$Duplicate_Count[i], times = replicate_count)
    
    # saving iteration
    replicate_vector = c(replicate_vector, replicate_iteration)
    
}

# adding to df 
count_mRNA_df$Replicate = replicate_vector
mRNA_df = count_mRNA_df %>%
    select(-Duplicate_Count)

head(mRNA_df)

Subject_No Subject_ID mRNA  Condensate Burn_Condition Concentration
1 6          M_6        HMOX1 PBS        PBS            NA           
2 6          M_6        HMOX1 PBS        PBS            NA           
3 6          M_6        HMOX1 PBS        PBS            NA           
4 6          M_6        HMOX1 PBS        PBS            NA           
5 6          M_6        HMOX1 PBS        PBS            NA           
6 6          M_6        HMOX1 PBS        PBS            NA           
  Time_Point Replicate ddCT        ddCT_pslog2
1 24         1          0.70848207 4.892803   
2 24         2          0.36651641 4.876100   
3 24         3         -0.09090465 4.853452   
4 24         4         -0.37654696 4.839126   
5 24         5          0.72283828 4.893500   
6 24         6          0.37135970 4.876338

Take the average ddCT to get rid of the replicates.

In [6]:
avg_mRNA_df = mRNA_df %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    mutate(Avg_ddCT = mean(ddCT)) %>% 
    select(-c("Replicate", "ddCT_pslog2", "ddCT")) %>%
    unique() %>%
    ungroup()

head(avg_mRNA_df)

Subject_No Subject_ID mRNA    Condensate Burn_Condition Concentration
1 6          M_6        HMOX1   PBS        PBS            NA           
2 5          F_5        ALDH3A1 PBS        PBS            NA           
3 5          F_5        CXCL1   PBS        PBS            NA           
4 5          F_5        CXCR1   PBS        PBS            NA           
5 5          F_5        GCLC    PBS        PBS            NA           
6 5          F_5        GCLM    PBS        PBS            NA           
  Time_Point Avg_ddCT   
1 24           0.2836241
2  4           0.4320455
3  4           0.6422482
4  4         -12.4016243
5  4           0.9108458
6  4           0.3423809

# Variance Stabilizing Transformation
This makes it so that the variance doesn't depend on the mean.
https://book.stat420.org/transformations.html

In [7]:
# first converting from long to wide format, which is necessary for the vsn function
wider_avg_mRNA_df = avg_mRNA_df %>%
# creating sample ids
    unite("SampleID", c(colnames(avg_mRNA_df)[c(1,4:7)]), sep = "_") %>%
    select(-Subject_ID) %>%
    pivot_wider(names_from = "SampleID", values_from = "Avg_ddCT") %>%
    column_to_rownames(var = "mRNA")

head(wider_avg_mRNA_df)

6_PBS_PBS_NA_24 5_PBS_PBS_NA_4 1_PBS_PBS_NA_4 2_PBS_PBS_NA_4
HMOX1     0.2836241       1.6220269      1.39023414     2.1102434   
ALDH3A1  -0.4184588       0.4320455      0.27473008     0.1204694   
CXCL1    -0.7070283       0.6422482      0.09744932     0.7277660   
CXCR1   -13.6424811     -12.4016243    -13.70600229   -13.5821411   
GCLC      0.1048251       0.9108458      0.90181502     0.6517629   
GCLM     -0.4755476       0.3423809      0.44173568     0.2984414   
        3_PBS_PBS_NA_4 4_PBS_PBS_NA_4 6_PBS_PBS_NA_4 6_PBS_PBS_NA_72
HMOX1     2.43685005     2.0919311      1.702591720    0.1381671    
ALDH3A1   0.07055185    -0.1803027      0.009163446   -1.0577533    
CXCL1     0.35592590     0.1674529      0.394399876    0.8012238    
CXCR1   -13.41281279   -13.2795531    -12.613186667  -12.4136794    
GCLC      0.74953458     0.6207409      0.653683036    0.1575451    
GCLM      0.36819982     0.2192093      0.007537574   -0.3875209    
        1_C_F_25_4   1_C_S_25_4   ⋯ 6_P_F_1_72  6_P_F_25_24 6_P_F_25_72 
HMOX1    -2.33804668  -3.22246963 ⋯   1.6515563 NA            0.22110064
ALDH3A1   0.16267528   0.41473799 ⋯  -0.4625463 NA           -1.07376220
CXCL1     0.03556161   1.09853908 ⋯  -1.8990085 NA           -0.02792455
CXCR1   -12.90925169 -14.10350485 ⋯ -15.0410890 NA          -13.34490982
GCLC     -0.31896658  -0.02863545 ⋯   0.3394122 NA           -0.82052995
GCLM      0.12643633   0.32477261 ⋯  -0.3960734 NA            0.15394273
        6_P_F_5_72  6_P_S_1_72  6_P_S_25_24 6_P_S_25_72 6_P_S_5_24  1_C_S_1_24 
HMOX1     0.7961182   0.8983407  -0.3778758   0.5217230  -0.1241551   0.2082109
ALDH3A1  -0.9375553  -1.3269869   0.4181893   1.7985573   0.2135130   0.4982387
CXCL1     3.8624448   3.3316444   1.0462532   0.6576701   0.7632177  -0.1058011
CXCR1   -11.8789158 -12.5749978 -11.9184997 -12.7449949 -11.8478948 -15.5025570
GCLC     -0.3441076  -1.1861843   0.1192901   0.8528297   0.1874478  -0.1877919
GCLM     -0.3305644  -0.1075840   0.3011212   0.4951484   0.2571859   0.3863060
        5_C_F_1_24 
HMOX1    -1.0969917
ALDH3A1   0.4459567
CXCL1    -0.5811714
CXCR1   -13.1482028
GCLC     -0.6381514
GCLM     -0.1976982

In [8]:
normalized_matrix = justvsn(as.matrix(wider_avg_mRNA_df), minDataPointsPerStratum = 19)
head(normalized_matrix)

6_PBS_PBS_NA_24 5_PBS_PBS_NA_4 1_PBS_PBS_NA_4 2_PBS_PBS_NA_4
HMOX1     3.912311        3.067190       3.843045       3.415457    
ALDH3A1   3.715852        2.605258       3.532961       2.692775    
CXCL1     3.626645        2.698362       3.476909       2.953958    
CXCR1   -20.295235      -19.997376     -20.310570     -20.264881    
GCLC      3.864761        2.809204       3.715390       2.923745    
GCLM      3.698636        2.563639       3.583843       2.774333    
        3_PBS_PBS_NA_4 4_PBS_PBS_NA_4 6_PBS_PBS_NA_4 6_PBS_PBS_NA_72 1_C_F_25_4
HMOX1     3.234524       3.294681       3.820872       3.703908      -16.250897
ALDH3A1   2.292604       2.443504       3.301868       3.237933        2.021796
CXCL1     2.444569       2.610919       3.437396       3.909343        1.919807
CXCR1   -20.153403     -20.079270     -20.230977     -20.401478      -20.385166
GCLC      2.630937       2.803508       3.521934       3.710346        1.590093
GCLM      2.450760       2.634254       3.301268       3.517304        1.993447
        1_C_S_25_4  ⋯ 6_P_F_1_72 6_P_F_25_24 6_P_F_25_72 6_P_F_5_72 6_P_S_1_72
HMOX1     0.7883619 ⋯   2.629532 NA          -17.17265    -1.229183 -17.74044 
ALDH3A1   3.6641076 ⋯   1.228801 NA          -18.06649   -17.570614 -18.61131 
CXCL1     3.8811991 ⋯ -13.877329 NA          -17.39302     2.873008 -14.32618 
CXCR1   -20.7294483 ⋯ -20.368879 NA          -20.49317   -20.586011 -20.32967 
GCLC      3.5035971 ⋯   1.926795 NA          -17.92993   -16.870849 -18.56933 
GCLM      3.6329506 ⋯   1.301355 NA          -17.23548   -16.850149 -18.19948 
        6_P_S_25_24 6_P_S_25_72 6_P_S_5_24  1_C_S_1_24 5_C_F_1_24
HMOX1     2.638051    2.409027  -14.4664678   2.094107   1.833391
ALDH3A1   3.065341    3.037754   -2.0847038   2.279381   2.853769
CXCL1     3.331071    2.490578    0.3527938   1.862415   2.259730
CXCR1   -20.410924  -20.407928  -20.2729677 -20.551241 -20.421350
GCLC      2.919338    2.600134   -2.4236255   1.795250   2.218369
GCLM      3.009904    2.392533   -1.6508262   2.210667   2.510943

# Subject Background Filter

Determining what subjects have missing data for all variables delineated based on the dfs the data will be split into and analyzed. Typically a subject is removed if it has less than 75% of data. 

In [9]:
subject_presence_df = wider_avg_mRNA_df %>%
    rownames_to_column(var = "mRNA") %>%
    pivot_longer(cols = 2:186, names_to = "Sample_ID", values_to = "Norm_ddCT") %>%
    separate(Sample_ID, into = c("Subject_No", "Condensate", "Burn_Condition", "Concentration", "Time_Point")) %>%
    mutate(count = ifelse(!is.na(Norm_ddCT), 1, 0)) %>%
    group_by(Time_Point, Concentration, Condensate, Burn_Condition, Subject_No) %>%
    summarize(Subject_Presence_Percentage = sum(count)/ length(unique(avg_mRNA_df$mRNA)) * 100) %>%
    arrange(Subject_Presence_Percentage)

subject_presence_df %>%
    filter(Subject_Presence_Percentage < 75)

`summarise()` has grouped output by 'Time_Point', 'Concentration',
'Condensate', 'Burn_Condition'. You can override using the `.groups` argument.


Time_Point Concentration Condensate Burn_Condition Subject_No
1  24         1             C          F              6         
2  24         1             C          S              4         
3  24         1             P          F              6         
4  24         1             P          S              4         
5  24         25            C          F              6         
6  24         25            C          S              4         
7  24         25            P          F              6         
8  24         5             C          F              6         
9  24         5             C          S              4         
10 24         5             P          F              5         
11 4          25            C          F              6         
12 72         1             C          F              6         
13 72         1             C          S              4         
14 72         1             P          S              4         
15 72         25            C          F              6         
16 72         25            C          S              4         
17 72         25            P          S              3         
18 72         5             C          F              6         
19 72         5             P          S              3         
20 72         NA            PBS        PBS            1         
21 72         NA            PBS        PBS            5         
   Subject_Presence_Percentage
1  15.78947                   
2  15.78947                   
3  15.78947                   
4  15.78947                   
5  15.78947                   
6  15.78947                   
7  15.78947                   
8  15.78947                   
9  15.78947                   
10 15.78947                   
11 15.78947                   
12 15.78947                   
13 15.78947                   
14 15.78947                   
15 15.78947                   
16 15.78947                   
17 15.78947                   
18 15.78947                   
19 15.78947                   
20 15.78947                   
21 15.78947

If we had the luxury of removing subjects, subjects 6,4,1,3, and 5 would simply be removed under all these conditions, since these subjects only have experimental values for 3 out of 19 mRNAs. However, in this case I think it's just best to impute to be able to perform later individual statistical analyses.

# mRNA Background Filter

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. Typically a variable is removed if it has less than 75% of data.

In [12]:
variable_presence_df = wider_avg_mRNA_df %>%
    rownames_to_column(var = "mRNA") %>%
    pivot_longer(cols = 2:186, names_to = "Sample_ID", values_to = "Norm_ddCT") %>%
    separate(Sample_ID, into = c("Subject_No", "Condensate", "Burn_Condition", "Concentration", "Time_Point")) %>%
    mutate(count = ifelse(!is.na(Norm_ddCT), 1, 0)) %>%
    group_by(Time_Point, Concentration, Condensate, Burn_Condition, mRNA) %>%
    # calculating the percentage of variables with data in all subjects
    summarize(Variable_Presence_Percentage = sum(count)/ length(unique(Subject_No)) * 100) %>%
    arrange(Variable_Presence_Percentage)

variable_presence_df %>%
    filter(Variable_Presence_Percentage < 75)

`summarise()` has grouped output by 'Time_Point', 'Concentration',
'Condensate', 'Burn_Condition'. You can override using the `.groups` argument.


Time_Point Concentration Condensate Burn_Condition mRNA   
1  24         NA            PBS        PBS            HMOX1  
2  24         1             C          F              GMCSF2 
3  24         1             C          F              IL6    
4  72         5             P          S              IL6    
5  72         NA            PBS        PBS            ALDH3A1
6  72         NA            PBS        PBS            CXCL1  
7  72         NA            PBS        PBS            CXCR1  
8  72         NA            PBS        PBS            GCLC   
9  72         NA            PBS        PBS            GCLM   
10 72         NA            PBS        PBS            GMCSF2 
11 72         NA            PBS        PBS            HMOX1  
12 72         NA            PBS        PBS            IL1B   
13 72         NA            PBS        PBS            IL6    
14 72         NA            PBS        PBS            IL8    
15 72         NA            PBS        PBS            MCP1   
16 72         NA            PBS        PBS            MUC5AC 
17 72         NA            PBS        PBS            MUC5B  
18 72         NA            PBS        PBS            NQO1   
19 72         NA            PBS        PBS            TBP    
20 72         NA            PBS        PBS            TXNRD1 
   Variable_Presence_Percentage
1  16.66667                    
2  66.66667                    
3  66.66667                    
4  66.66667                    
5  66.66667                    
6  66.66667                    
7  66.66667                    
8  66.66667                    
9  66.66667                    
10 66.66667                    
11 66.66667                    
12 66.66667                    
13 66.66667                    
14 66.66667                    
15 66.66667                    
16 66.66667                    
17 66.66667                    
18 66.66667                    
19 66.66667                    
20 66.66667

Based on this, *HMOX1* should be removed from control samples at 24 hours exposure along with *GMCSF2* and *IL6* under their respective conditions. However, for aforementioned reasons they will be retained.

# QRILC Imputation

In [9]:
preimputed_df = data.frame(normalized_matrix)
imputed_QRILC_object = impute.QRILC(preimputed_df, tune.sigma = 0.1)
imputed_df = data.frame(imputed_QRILC_object[1]) %>%
    rownames_to_column(var = "mRNA") %>%
    pivot_longer(cols = 2:186, names_to = "Sample_ID", values_to = "Norm_ddCT") %>%
    separate(Sample_ID, into = c("Subject_No", "Condensate", "Burn_Condition", "Concentration", "Time_Point"))

# dataframes automatically add X to col names if it starts with a number so removing that here
imputed_df$Subject_No = as.numeric(gsub("X","", as.character(imputed_df$Subject_No)))

# adding back in subject ids
final_df = full_join(imputed_df, unique(avg_mRNA_df %>%
                        select(c("Subject_No", "Subject_ID"))))

# rearranging cols
final_df = final_df[,c(2,8,1,3:7)]
head(final_df)

Joining with `by = join_by(Subject_No)`


Subject_No Subject_ID mRNA  Condensate Burn_Condition Concentration
1 6          M_6        HMOX1 PBS        PBS            NA           
2 5          F_5        HMOX1 PBS        PBS            NA           
3 1          F_1        HMOX1 PBS        PBS            NA           
4 2          M_2        HMOX1 PBS        PBS            NA           
5 3          M_3        HMOX1 PBS        PBS            NA           
6 4          F_4        HMOX1 PBS        PBS            NA           
  Time_Point Norm_ddCT
1 24         3.912311 
2 4          3.067190 
3 4          3.843045 
4 4          3.415457 
5 4          3.234524 
6 4          3.294681

In [10]:
# original data with replicates
dim(mRNA_df)

# after averaging replicates
dim(avg_mRNA_df)

# adding in missing data for remaining records and imputing
dim(final_df)

[1] 4211   10

[1] 3162    8

[1] 3515    8

In [13]:
# percentage of total records that weren't imputed
3162/3515

[1] 0.8995733

In [14]:
# percentage of records that weren't imputed that were used in downstream analyses
avg_mRNA_df %>%
    filter(Time_Point != 72, Concentration != 5) %>%
    group_by(Time_Point, Condensate, Burn_Condition, Concentration) %>%
    count()

1452/1596

Time_Point Condensate Burn_Condition Concentration n  
1   4         C          F              25             95
2   4         C          S              25            114
3   4         P          F              25            114
4   4         P          S              25            114
5   4         PBS        PBS            NA            114
6  24         C          F              1              93
7  24         C          F              25             98
8  24         C          S              1              95
9  24         C          S              25             98
10 24         P          F              1              98
11 24         P          F              25             98
12 24         P          S              1              98
13 24         P          S              25            114
14 24         PBS        PBS            NA            109

[1] 0.9097744

In [12]:
# exporting
write.xlsx(final_df, paste0(Output,"/", "Processed_mRNA_Data_", cur_date, ".xlsx"), rowNames = FALSE)